# The sound of Silence


### Dependencias
* https://github.com/jiaaro/pydub ou pip3 install pydub
* sudo apt install ffmpeg
* sudo apt install python3-pip
* pip3 install keras
* pip3 install tensorflow
* pip3 install pandas
* pip3 install h5py

# Note
    This produre start using a module data, but the code get to messy and I start to code a small "system" for manage the research, thus the cell's that import data are kind of deprecated.
 

In [1]:
datasetMusan = "/home/joseildo/codigos/voiceDetection/musan/"
datasetMusanBalanced6percente = "/home/joseildo/codigos/voiceDetection/Bmusan"
datasetCodeTest = "/home/joseildo/codigos/voiceDetection/dataCodeTest/"
%store datasetMusan
%store datasetMusanBalanced6percente
%store datasetCodeTest

Stored 'datasetMusan' (str)
Stored 'datasetMusanBalanced6percente' (str)
Stored 'datasetCodeTest' (str)


In [ ]:
import data
from keras.models import Sequential
from keras.layers import Dense,Dropout

## pastas
audios = "/home/joseildo/codigos/voiceDetection/musan/"

data.iterateOnFold(audios, shuffle = True)

## modelo de teste
modelo_um = Sequential()
modelo_um.add(Dense(400, input_shape=(None,1600)))
modelo_um.add(Dense(400,activation="sigmoid"))
modelo_um.add(Dense(200,activation="sigmoid"))
modelo_um.add(Dropout(0.2))
modelo_um.add(Dense(100,activation="sigmoid"))
modelo_um.add(Dense(4,activation = 'softmax'))
modelo_um.compile(loss='categorical_crossentropy',optimizer = 'sgd',metrics=['acc'])

while(data.hasNext()):
    x,y = data.next()
    modelo_um.fit(x,y,nb_epoch = 1)
    modelo_um.save_weights(audios+"pounds_dense.h5")
    

In [ ]:
import data
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras import metrics
## pastas
audios = "/home/gtad/gapsSom/voiceDetection/musan"

data.iterateOnFold(audios, shuffle = True)

## modelo de teste
modelo_2 = Sequential()
modelo_2.add(Dense(200,activation="sigmoid", input_shape=(None,1600)))
modelo_2.add(Dense(100,activation="sigmoid"))
modelo_2.add(Dense(4,activation = 'softmax'))
modelo_2.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics=[metrics.categorical_accuracy])
modelo_2.save("model_three.h5")

while(data.hasNext()):
    x,y = data.next()
    modelo_2.fit(x,y,epochs = 1)
    modelo_2.save_weights("/home/gtad/gapsSom/voiceDetection/pounds_conv.h5")

In [ ]:
import data
from keras.models import Sequential
from keras.layers import Convolution1D,Dropout,Dense, MaxPooling1D, Activation
from keras import metrics
## pastas
audios = "/home/gtad/gapsSom/voiceDetection/musan/"

data.iterateOnFold(audios, shuffle = True)

print("modelo")
## modelo de teste
modelo_3 = Sequential()
modelo_3.add(Convolution1D(100, 50, padding="same", input_shape=(None,1600)))
modelo_3.add(MaxPooling1D(50,padding="same"))
modelo_3.add(Activation("relu"))
modelo_3.add(Convolution1D(50, 5, padding="same"))
modelo_3.add(MaxPooling1D(50,padding="same"))
modelo_3.add(Activation("relu"))
modelo_3.add(Dense(50))
modelo_3.add(Activation("tanh"))
modelo_3.add(Dropout(0.5))
modelo_3.add(Dense(4))
modelo_3.add(Activation("softmax"))
modelo_3.compile(loss='categorical_crossentropy',optimizer = 'sgd',metrics=["acc"])
modelo_3.save("cov_1.h5")
print(modelo_3.summary())
print("compilado")

while(data.hasNext()):
    x,y = data.next()
    x = x.reshape(x.shape[1],1,x.shape[2])
    y = y.reshape(y.shape[1],1,y.shape[2])    
    modelo_3.fit(x,y,epochs = 1)
    modelo_3.save_weights("/home/gtad/gapsSom/voiceDetection/pounds_conv.h5")

## Statistics
### Todo-List
* Produzir 20% de silencio
* Produzir ruido
* Gerar musicas misturadas com fala
* Separar os silencio dos audios
* Testes
    * testar usando 10% de silencio mais 10% de musica sem voz contra os 20% de fala
    * testar usando 10% de silencio mais 10% de musica com voz contra 20% de fala
    * testar usando (20/3)% de silencio mais (20/3)% de musica sem voz mais (20/3)% ruido contra 20% de fala    
>> fma contains voice

In [1]:
from keras.models import load_model
import data

audios = "/home/gtad/gapsSom/voiceDetection/musan/"
data.iterateOnFold(audios)

silence_quant = 0
speech_quant = 0
music_quant = 0
noise_quant = 0

while(data.hasNext()):
    try:
        x,y = data.next()
        for i in y[0]:
            if i[0] == 1:
                silence_quant += 1
            elif i[1] == 1:
                speech_quant += 1
            elif i[2] == 1:
                music_quant += 1
            elif i[3] == 1:
                noise_quant += 1                
    except:
        pass
count = silence_quant + speech_quant + music_quant + noise_quant
print("\nsilence percentage: %.2f"%(silence_quant/count))
print("speech percentage: %.2f"%(speech_quant/count))
print("music percentage: %.2f"%(music_quant/count))
print("noise percentage: %.2f"%(noise_quant/count))

Using TensorFlow backend.


[============================================================] 100.0% ...loading and running fold
silence percentage: 0.06
speech percentage: 0.51
music percentage: 0.38
noise percentage: 0.05


In [9]:
from pydub import AudioSegment as AS
from pydub import silence as dub
from pydub.utils import mediainfo
import os
music = 0
mC = 0
noise = 0
nC = 0
speech = 0
sC = 0

os.chdir("/home/joseildo/codigos/voiceDetection/musan/")
for audio in os.listdir():
    sound = mediainfo(audio)
    if "speech" in audio:
        speech += float(sound["duration"])
        sC += 1
    elif "music" in audio:
        music += float(sound["duration"])
        mC += 1
    else:
        noise += float(sound["duration"])
        nC += 1
        
time = music + noise + speech
total = mC + nC + sC
print("music time:",music,"noise time:",noise,"speech time:",speech,"\n")
print("music quant:",mC,"noise quant:",nC,"speech quant:",sC)
print("total time:",time)
print("music percentage of seconds: %.2f"%(music/time))
print("noise percentage of seconds: %.2f"%(noise/time))
print("speech percentage of seconds: %.2f"%(speech/time))
print("music percentage of files: %.2f"%(mC/total))
print("noise percentage of files: %.2f"%(nC/total))
print("speech percentage of files: %.2f"%(sC/total))

music time: 153412.7663940001 noise time: 22414.717253999992 speech time: 217615.099637 

music quant: 660 noise quant: 930 speech quant: 426
total time: 393442.58328500006
music percentage of seconds: 0.39
noise percentage of seconds: 0.06
speech percentage of seconds: 0.55
music percentage of files: 0.33
noise percentage of files: 0.46
speech percentage of files: 0.21


### Slip Silence from sound

In [ ]:
from Folder import Folder
from Data import Data

folder = Folder(src =  datasetCodeTest)

with Data(folder = folder) as data:
    folder.regex = "(?=music)"
    for audio in iter(data):
        audio.splitSilence().appendSilences()
        audio.splitSound().appendSounds()
    audio.storeSounds(name="music")
    
    folder.regex = "(?=noise)"
    for audio in iter(data):
        audio.splitSilence().appendSilences()
        audio.splitSound().appendSounds()
    audio.storeSounds(name="noise")
    
    folder.regex = "(?=speech)"
    for audio in iter(data):
        audio.splitSilence().appendSilences()
        audio.splitSound().appendSounds()
    audio.storeSounds(name="speech")
    
    audio.storeSilences(name="silence")
    